In [141]:
import os
from pathlib import Path
import src.download_yf as dlyf
import src.preprocessing as prep
import importlib
import pandas as pd
import src.arima_train as arima_train
from statsmodels.tsa.arima.model import ARIMAResults

cwd = os.getcwd()
project_root=Path(cwd).parent
data_dir = project_root / "data"
models_dir = project_root / "models"

##  I ) --------------------- Test >>  src.download_yf 

In [142]:
importlib.reload(dlyf)
tidy_all = dlyf.update_fchi_tidy()
tidy_all[tidy_all["ticker"] == "^FCHI"].tail()


1 Failed download:
['^FCHI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-11-22 -> 2025-11-24)')


[INFO] Tidy existant chargé : /Users/souhail/projets-ml/indexpulse/data/raw/ohlcv_10y.parquet
[INFO] Dernière date FCHI dans le parquet tidy : 2025-11-21 00:00:00
[INFO] Téléchargement FCHI du 2025-11-22 au 2025-11-24…
[WARN] yf.download a renvoyé vide pour ['^FCHI'] entre 2025-11-22 et 2025-11-24
[WARN] Aucunes nouvelles lignes reçues pour FCHI.


,date,ticker,open,high,low,close,adj_close,volume
2556,2025-11-17,^FCHI,8155.930176,8176.100098,8109.819824,8119.020020,8119.020020,42960200.0
2557,2025-11-18,^FCHI,8017.729980,8032.509766,7925.060059,7967.930176,7967.930176,68564200.0
2558,2025-11-19,^FCHI,7964.770020,8005.750000,7919.359863,7953.770020,7953.770020,55042800.0
2559,2025-11-20,^FCHI,8044.850098,8057.700195,7981.069824,7981.069824,7981.069824,60357000.0
2560,2025-11-21,^FCHI,7881.100098,7996.459961,7875.870117,7982.649902,7982.649902,64784800.0


## II ) -------------------------- Test >>  src.preprocessing.py 

In [143]:
importlib.reload(prep)
feat_fchi = prep.preprocess_cac40_from_tidy(tidy_all)
feat_fchi.tail(), feat_fchi.shape

[OK] Features FCHI écrites dans : /Users/souhail/projets-ml/indexpulse/data/processed/FCHI_features_h1d.parquet


(           date ticker         open         high          low        close  \
 2355 2025-11-14  ^FCHI  8186.140137  8202.480469  8084.229980  8170.089844   
 2356 2025-11-17  ^FCHI  8155.930176  8176.100098  8109.819824  8119.020020   
 2357 2025-11-18  ^FCHI  8017.729980  8032.509766  7925.060059  7967.930176   
 2358 2025-11-19  ^FCHI  7964.770020  8005.750000  7919.359863  7953.770020   
 2359 2025-11-20  ^FCHI  8044.850098  8057.700195  7981.069824  7981.069824   
 
         adj_close      volume  dow  month  ...       ema200  volstd200  \
 2355  8170.089844  54654300.0    4     11  ...  7840.150927   0.010450   
 2356  8119.020020  42960200.0    0     11  ...  7842.925744   0.010459   
 2357  7967.930176  68564200.0    1     11  ...  7844.169569   0.010492   
 2358  7953.770020  55042800.0    2     11  ...  7845.260121   0.010488   
 2359  7981.069824  60357000.0    3     11  ...  7846.611461   0.010487   
 
       vol_sma200    vol_lag1    vol_lag2    vol_lag5   vol_lag10       

## III ) ---------------------- Test >>> arima_train.py

In [144]:
importlib.reload(arima_train)
metrics = arima_train.train_arima_logret1()
metrics

Split temporel : train=1653, val=354, test=353, n_dates=2360
Séries : train=1653, val=354, test=353
ARIMA VAL RMSE=0.008252   MAE=0.006257
ARIMA TEST RMSE=0.009796   MAE=0.007219
Modèle ARIMA sauvegardé sous : /Users/souhail/projets-ml/indexpulse/models/arima_logret1_FCHI.pkl


/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is availab

{'rmse_val': 0.008251699693177503,
 'mae_val': 0.006257279383915757,
 'rmse_test': 0.009796097597593959,
 'mae_test': 0.007218910492449248,
 'model_path': PosixPath('/Users/souhail/projets-ml/indexpulse/models/arima_logret1_FCHI.pkl')}

## IV ) --------- Model summary 

In [145]:
model_path = models_dir/"arima_logret1_FCHI.pkl"
model = ARIMAResults.load(model_path)
print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                logret1   No. Observations:                 2007
Model:                 ARIMA(1, 0, 1)   Log Likelihood                6139.108
Date:                Mon, 24 Nov 2025   AIC                         -12270.215
Time:                        18:11:11   BIC                         -12247.798
Sample:                             0   HQIC                        -12261.985
                               - 2007                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.000      0.982      0.326      -0.000       0.001
ar.L1         -0.0034      1.186     -0.003      0.998      -2.329       2.322
ma.L1         -0.0062      1.190     -0.005      0.9

# ----------------------------
# ----------------------------
# ----------------------------
# ----------------------------
# ----------------------------
# ----------------------------
# ----------------------------
# ----------------------------

## 1) Téléchargements : Appel multi-tickers 

## 2) Format Tidy

##  3) Feature engineering 

#### Fonctions utilitaires 

TICKERS = ["^GSPC", "^FCHI", "^STOXX50E"]  # déjà téléchargés dans `tidy`

# Fenêtres & lags de base (ajuste si besoin)
FE_WINDOWS = [5, 10, 20, 50, 100, 200]
FE_LAGS = [1, 2, 5, 10]
HORIZON_D = 1  # on prédit le retour à J+1

def add_calendar_features(df: pd.DataFrame) -> pd.DataFrame:
    di = df["date"]
    df["dow"] = di.dt.dayofweek        # 0=Mon ... 4=Fri
    df["month"] = di.dt.month
    # .is_month_end / .is_quarter_end
    df["is_month_end"] = di.dt.is_month_end.astype(int)
    df["is_quarter_end"] = di.dt.is_quarter_end.astype(int)
    # semaine ISO et jour de l'année (optionnel)
    iso = di.dt.isocalendar()
    df["weekofyear"] = iso.week.astype(int)
    df["dayofyear"] = di.dt.dayofyear
    return df

def add_return_features(df: pd.DataFrame) -> pd.DataFrame:
    # retours basés sur CLOSE
    df["ret1"] = df["close"].pct_change()
    df["logret1"] = np.log(df["close"] / df["close"].shift(1))
    # lags de retours
    for L in FE_LAGS:
        df[f"ret_lag{L}"] = df["ret1"].shift(L)
        df[f"logret_lag{L}"] = df["logret1"].shift(L)
    return df

def add_ma_vol_features(df: pd.DataFrame) -> pd.DataFrame:
    # SMA/EMA sur CLOSE + volatilité (std des retours) + volume laggé
    for w in FE_WINDOWS:
        df[f"sma{w}"] = df["close"].rolling(w, min_periods=w).mean()
        df[f"ema{w}"] = df["close"].ewm(span=w, adjust=False, min_periods=w).mean()
        df[f"volstd{w}"] = df["ret1"].rolling(w, min_periods=w).std()
        df[f"vol_sma{w}"] = df["volume"].rolling(w, min_periods=w).mean()
    for L in FE_LAGS:
        df[f"vol_lag{L}"] = df["volume"].shift(L)
    return df

def add_atr(df: pd.DataFrame, n: int = 14) -> pd.DataFrame:
    # True Range = max(high-low, |high-prev_close|, |low-prev_close|)
    prev_close = df["close"].shift(1)
    tr = pd.concat([
        (df["high"] - df["low"]).abs(),
        (df["high"] - prev_close).abs(),
        (df["low"] - prev_close).abs()
    ], axis=1).max(axis=1)
    df["atr"] = tr.rolling(n, min_periods=n).mean()
    return df

def make_target(df: pd.DataFrame, horizon: int = 1) -> pd.DataFrame:
    # target = retour futur sur CLOSE
    df["target_ret"] = df["close"].pct_change(horizon).shift(-horizon)
    df["target_logret"] = (np.log(df["close"] / df["close"].shift(horizon))).shift(-horizon)
    return df

def build_features_one_ticker(tidy_one: pd.DataFrame, horizon: int = 1) -> pd.DataFrame:
    df = tidy_one.sort_values("date").copy()
    df = add_calendar_features(df)
    df = add_return_features(df)
    df = add_ma_vol_features(df)
    df = add_atr(df, n=14)
    df = make_target(df, horizon=horizon)

    # Nettoyage
    need_cols = ["ret1", "logret1", "atr", "target_ret"]
    need_cols += [f"sma{w}" for w in FE_WINDOWS] + [f"volstd{w}" for w in FE_WINDOWS]
    df = df.dropna(subset=need_cols).reset_index(drop=True)

    # ⚠️ Pas de standardisation ici : on garde les vraies échelles
    # (on pourra scaler dynamiquement plus tard dans le pipeline de modélisation)

    return df

features_by_ticker = {}
for t in TICKERS:
    one = tidy[tidy["ticker"] == t].copy()
    feat = build_features_one_ticker(one, horizon=HORIZON_D)
    features_by_ticker[t] = feat
    print(t, feat.shape, "de", one.shape)

# Exemples d’aperçu
features_by_ticker["^GSPC"].head()


df = df_cac.copy()
df = df.sort_values("date")


# dates uniques triées
unique_dates = df["date"].sort_values().unique()
n_dates = len(unique_dates)
train_end = unique_dates[int(0.7 * n_dates)]   # ~70% des dates
val_end   = unique_dates[int(0.85 * n_dates)]  # ~15% val, ~15% test

train = df[df["date"] <= train_end]
val   = df[(df["date"] > train_end) & (df["date"] <= val_end)]
test  = df[df["date"] > val_end]

len(train), len(val), len(test), n_dates

feature_cols = [
    c for c in df.columns
    if c not in ("date", "ticker", "target_ret", "target_logret")
]

X_train, y_train = train[feature_cols], train["target_logret"]
X_val,   y_val   = val[feature_cols],   val["target_logret"]
X_test,  y_test  = test[feature_cols],  test["target_logret"]

X_train.shape, X_val.shape, X_test.shape


from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

def eval_metrics(y_true, y_pred, name=""):
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae  = float(mean_absolute_error(y_true, y_pred))
    print(f"{name} RMSE={rmse:.6f}  MAE={mae:.6f}")
    return rmse, mae

model_xgb = XGBRegressor(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
)


model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False,
)

model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False,
)

pred_val  = model_xgb.predict(X_val)
pred_test = model_xgb.predict(X_test)

rmse_val, mae_val   = eval_metrics(y_val,  pred_val,  "XGB VAL")
rmse_test, mae_test = eval_metrics(y_test, pred_test, "XGB TEST")


from statsmodels.tsa.arima.model import ARIMA

# série de logret1 indexée par date
series = df.set_index("date")["logret1"].sort_index()

train_s = series[series.index <= train_end].dropna()
val_s   = series[(series.index > train_end) & (series.index <= val_end)].dropna()
test_s  = series[series.index > val_end].dropna()

len(train_s), len(val_s), len(test_s)




# ARIMA sur train, prévision sur val
arima_model = ARIMA(train_s, order=(1, 0, 1))
arima_res   = arima_model.fit()

fc_val = arima_res.forecast(steps=len(val_s))
fc_val.index = val_s.index  # aligner les index

rmse_val_arima, mae_val_arima = eval_metrics(val_s, fc_val, "ARIMA VAL")

# Refit sur train+val, prévision sur test
trainval_s = series[series.index <= val_end].dropna()
arima_model_tv = ARIMA(trainval_s, order=(1,0,1))
arima_res_tv   = arima_model_tv.fit()

fc_test = arima_res_tv.forecast(steps=len(test_s))
fc_test.index = test_s.index

rmse_test_arima, mae_test_arima = eval_metrics(test_s, fc_test, "ARIMA TEST")


from prophet import Prophet

df_prophet = (
    df[["date", "logret1"]]
    .rename(columns={"date": "ds", "logret1": "y"})
    .dropna()
)

train_p = df_prophet[df_prophet["ds"] <= train_end]
val_p   = df_prophet[(df_prophet["ds"] > train_end) & (df_prophet["ds"] <= val_end)]
test_p  = df_prophet[df_prophet["ds"] > val_end]

m = Prophet(
    weekly_seasonality=True,
    yearly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode="additive",
)
m.fit(train_p)

# Prévisions sur la validation
fc_val_p = m.predict(val_p[["ds"]])
pred_val_p = fc_val_p["yhat"].values

rmse_val_prophet, mae_val_prophet = eval_metrics(val_p["y"], pred_val_p, "Prophet VAL")

# Refit sur train+val, prévisions sur test
trainval_p = df_prophet[df_prophet["ds"] <= val_end]
m_tv = Prophet(
    weekly_seasonality=True,
    yearly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode="additive",
)
m_tv.fit(trainval_p)

fc_test_p = m_tv.predict(test_p[["ds"]])
pred_test_p = fc_test_p["yhat"].values

rmse_test_prophet, mae_test_prophet = eval_metrics(test_p["y"], pred_test_p, "Prophet TEST")


print(df["logret1"].std())
print(df["logret1"].mean())


### Sauvegarde modèle ARIMA 

from pathlib import Path

project_root = Path.cwd().parent          # ../ depuis notebooks/

models_dir.mkdir(exist_ok=True)

arima_path = models_dir / "arima_logret1_FCHI.pkl"
arima_res_tv.save(arima_path)

print("ARIMA sauvegardé sous", arima_path)

